In [5]:
from util.sys import get_path
import pandas as pd
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from extract_graph import (
    generate_nx_graph,
    transform_list,
    generate_skeleton,
    generate_nx_graph_from_skeleton,
    from_connection_tab,
)
from node_id import whole_movement_identification, second_identification
import ast
from plotutil import plot_t_tp1, compress_skeleton
from scipy import sparse
from sparse_util import dilate, zhangSuen
from realign import realign
from datetime import datetime, timedelta
from node_id import orient
import pickle
from matplotlib.widgets import CheckButtons
import scipy.io as sio
import imageio
from pymatreader import read_mat
import os
from experiment_class import Experiment
import cv2

In [2]:
plate = 13
directory = (
    "//sun.amolf.nl/shimizu-data/home-folder/oyartegalvez/Drive_AMFtopology/PRINCE"
)
listdir = os.listdir(directory)
list_dir_interest = [
    name
    for name in listdir
    if name.split("_")[-1] == f'Plate{0 if plate<10 else ""}{plate}'
]
ss = [name.split("_")[0] for name in list_dir_interest]
ff = [name.split("_")[1] for name in list_dir_interest]
dates_datetime = [
    datetime(
        year=int(ss[i][:4]),
        month=int(ss[i][4:6]),
        day=int(ss[i][6:8]),
        hour=int(ff[i][0:2]),
        minute=int(ff[i][2:4]),
    )
    for i in range(len(list_dir_interest))
]
dates_datetime.sort()
dates_datetime_chosen = dates_datetime[1:25]
dates = [
    f'{0 if date.month<10 else ""}{date.month}{0 if date.day<10 else ""}{date.day}_{0 if date.hour<10 else ""}{date.hour}{0 if date.minute<10 else ""}{date.minute}'
    for date in dates_datetime_chosen
]

In [3]:
exp = Experiment(13)
exp.load(dates, local=False, raw=False)

In [4]:
tip_track = {}
for node in exp.nodes:
    for t in range(len(exp.dates)):
        if node.is_in(t) and node.degree(t) == 1:
            track = []
            for tsup in range(t, len(exp.dates)):
                if node.is_in(tsup):
                    track.append(node.pos(tsup))
            tip_track[node.label] = track
            break
tip_growth = {
    tip: [
        np.linalg.norm(tip_track[tip][i + 1] - tip_track[tip][i])
        for i in range(len(tip_track[tip]) - 1)
    ]
    for tip in tip_track.keys()
}
tip_of_interest = [
    tip for tip in tip_growth.keys() if np.any(np.array(tip_growth[tip]) >= 100)
]

In [ ]:
tip = choice(tip_of_interest)
node_interest = tip
poss = exp.positions
ts = [t for t in range(len(poss)) if exp.get_node(tip).is_in(t)]

In [ ]:
tip = choice(tip_of_interest)
node_interest = tip
poss = exp.positions
ts = [t for ti in range(len(poss)) if exp.get_node(tip).is_in(t)]
pos_problem = [poss[i][node_interest] for i in ts]
xbegin = [pos_problem[i][0] - 1500 for i in ts]
ybegin = [pos_problem[i][1] - 1500 for i in ts]
xend = [pos_problem[i][0] + 1500 for i in ts]
yend = [pos_problem[i][1] + 1500 for i in ts]
skeletons_small = []
for i in ts:
    skeletons_small.append(exp.skeletons[i][xbegin[i] : xend[i], ybegin[i] : yend[i]])
node_smalls = []
for i in ts:
    node_smalls.append(
        [
            node
            for node in exp.nx_graph[i].nodes
            if (
                xbegin[i] < poss[i][node][0] < xend[i]
                and ybegin[i] < poss[i][node][1] < yend[i]
                and nx_graph.degree(node) >= 1
            )
        ]
    )
kernel = np.ones((5, 5), np.uint8)
skeletons_small_dilated = [
    cv2.dilate(skeleton.todense().astype(np.uint8), kernel, iterations=1)
    for skeleton in skeletons_small
]
for tp1 in range(len(growths)):
    plot_t_tp1(
        node_smalls[tp1],
        node_smalls[tp1],
        poss[tp1],
        poss[tp1],
        skeletons_small_dilated[tp1],
        skeletons_small_dilated[tp1],
        shift=(xbegin[tp1], ybegin[tp1]),
        save=f"Data/video_test/network_timestep_{tp1}",
        time=f"t={4*tp1}h",
    )
images = []
for t in range(len(growths)):
    images.append(imageio.imread(f"Data/video_test/network_timestep_{t}.png"))
imageio.mimsave(f"Data/video_test/{node_interest}movie_track.gif", images, duration=1)